In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2

from matplotlib import pylab as plt
import psdio

import warnings
warnings.filterwarnings("ignore")

# Example 31

In [ ]:
psd_path     = "sample_31.psd"
psd_npy      = psdio.large_psd_to_npy(psd_path)
image, cells = psdio.get_masks_from_layer(psd_npy)

image        = image[:,:,0].astype(float)
image        = image/image.max()

Unknown image resource 1092
Unknown key: b'CAI '
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =144'


In [ ]:
image.shape, cells.shape ### one layer per cell -> Nr. of cells = Nr. of layers

In [ ]:
instance  = np.sum((cells.T*np.arange(1,len(cells)+1).T).T,0)
semantics = instance.astype(bool)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(131)
plt.imshow(image,"Greys_r",vmin=0.4,vmax=0.6)
plt.axis("off")
plt.title("image", loc="left")
plt.subplot(132)
plt.imshow(instance)
plt.axis("off")
plt.title("instance mask", loc="left")
plt.subplot(133)
plt.imshow(semantics)
plt.axis("off")
plt.title("segmentation mask", loc="left")
plt.show()

# Detection 
## Rule/filter based detection

In [ ]:
from detection import detect_cells

In [ ]:
mask = detect_cells(image,edge_sigma=3.5,obsize=500)

plt.figure(figsize=(12,7))
plt.subplot(131)
plt.imshow(image,"Greys",vmin=0.4,vmax=0.6)
plt.colorbar(shrink=0.25)
plt.axis("off")
plt.title("Sobel edges", loc="left")

plt.subplot(132)
plt.imshow(mask)
plt.colorbar(shrink=0.25)
plt.axis("off")
plt.title("refined Mask", loc="left")

plt.subplot(133)
plt.imshow(semantics)
plt.colorbar(shrink=0.25)
plt.axis("off")
plt.title("segmentation mask", loc="left")
plt.show()

In [ ]:
import torch
import segmentation_models_pytorch as smp

In [ ]:
tp, fp,fn, tn = smp.metrics.get_stats(torch.tensor(mask), 
                                      torch.tensor(semantics), 
                                      mode='binary', threshold=0.5,)

plt.figure(figsize=(12,3))
names = ["TP", "FP", "FN", "TN"]
for n,i in enumerate([tp, fp, fn, tn]):
    plt.subplot(141+n)
    plt.imshow(i)
    plt.axis("off")
    plt.title(names[n], loc="left")
plt.show()

In [ ]:
IOU       = smp.metrics.iou_score(tp, fp, fn, tn)
IOU_micro = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
IOU_macro = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")

precision = smp.metrics.precision(tp, fp, fn, tn,reduction="micro")
accuracy  = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
f1_score  = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
f2_score  = smp.metrics.fbeta_score(tp, fp, fn, tn, beta=2, reduction="micro")
recall    = smp.metrics.recall(tp, fp, fn, tn, reduction="micro-imagewise")

print("IOU: "      , float(IOU_micro),float(IOU_macro))
print("precision: ", float(precision))
print("f1_score: " , float(f1_score))
print("f2_score: " , float(f2_score))
print("accuracy: " , float(accuracy))
print("recall: "   , float(recall))

# Example 16

In [ ]:
psd_path     = "sample_16.psd"
psd_npy      = psdio.large_psd_to_npy(psd_path)
image, cells = psdio.get_masks_from_layer(psd_npy)

instance  = np.sum((cells.T*np.arange(1,len(cells)+1).T).T,0)
semantics = instance.astype(bool)

image        = image[:,:,0].astype(float)
image        = image/image.max()

mask         = detect_cells(image,edge_sigma=1.5,obsize=2000)

plt.figure(figsize=(12,7))
plt.subplot(131)
plt.imshow(image)
plt.colorbar(shrink=0.25)
plt.axis("off")
plt.title("Sobel edges", loc="left")

plt.subplot(132)
plt.imshow(mask)
plt.colorbar(shrink=0.25)
plt.axis("off")
plt.title("refined Mask", loc="left")

plt.subplot(133)
plt.imshow(semantics)
plt.colorbar(shrink=0.25)
plt.axis("off")
plt.title("segmentation mask", loc="left")
plt.show()

In [ ]:
tp, fp,fn, tn = smp.metrics.get_stats(torch.tensor(mask), 
                                      torch.tensor(semantics), 
                                      mode='binary', threshold=0.5,)

plt.figure(figsize=(12,3))
names = ["TP", "FP", "FN", "TN"]
for n,i in enumerate([tp, fp, fn, tn]):
    plt.subplot(141+n)
    plt.imshow(i)
    plt.axis("off")
    plt.title(names[n], loc="left")
plt.show()

In [ ]:
IOU       = smp.metrics.iou_score(tp, fp, fn, tn)
IOU_micro = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")
IOU_macro = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")

precision = smp.metrics.precision(tp, fp, fn, tn,reduction="micro")
accuracy  = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro")
f1_score  = smp.metrics.f1_score(tp, fp, fn, tn, reduction="micro")
f2_score  = smp.metrics.fbeta_score(tp, fp, fn, tn, beta=2, reduction="micro")
recall    = smp.metrics.recall(tp, fp, fn, tn, reduction="micro-imagewise")

print("IOU: "      , float(IOU_micro),float(IOU_macro))
print("precision: ", float(precision))
print("f1_score: " , float(f1_score))
print("f2_score: " , float(f2_score))
print("accuracy: " , float(accuracy))
print("recall: "   , float(recall))

